<a href="https://colab.research.google.com/github/JorgeLMF/cc42_tf_202014828_201915747_201921559_201811355/blob/main/Trabajo_Final_Hito_3_Elena_Salcedo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ALGORITMO ELENA SALCEDO

##IMPORTS

In [4]:
import graphviz as gv
import numpy as np
import pandas as pd
import heapq as hq
import time
import math

In [5]:
url = "https://raw.githubusercontent.com/JorgeLMF/cc42_tf_202014828_201915747_201921559_201811355_201714378/main/DATASET/almacenesXyY.csv"
url2 = "https://raw.githubusercontent.com/JorgeLMF/cc42_tf_202014828_201915747_201921559_201811355_201714378/main/DATASET/puntosentregaXyY.csv"
file_almacenes = pd.read_csv(url).to_numpy()
pts_entrega = pd.read_csv(url2).to_numpy()

##ELABORACIÓN

###LA_MATRIX

In [68]:
la_matrix = []
for i in range(1000):
    l = [0]*1000                          # N°0 -> ni almacén ni punto de entrega
    la_matrix.append(l)

for line in file_almacenes:
    la_matrix[line[0]][line[1]] = 1       # N°1 -> almacenes

for line in pts_entrega:
    if la_matrix[line[0]][line[1]] != 1:
       la_matrix[line[0]][line[1]] = 2    # N°2 -> puntos de entrega

In [69]:
def mostrar_matriz():
  for line in la_matrix:
    print(line)

###PUNTOS

In [70]:
puntos = list()
n = 1000
for i in range(n*n):
    l = [0]*4
    puntos.append(l)

for i in range(n):
  for j in range(n):
    puntos[j+n*i][0] = j                # 0 = eje x
    puntos[j+n*i][1] = i                # 1 = eje y
    puntos[j+n*i][2] = la_matrix[i][j]  # 2 = tipo de punto
    puntos[j+n*i][3] = -1               # 3 = padre

###GRAFO

In [71]:
grafo = list()

for i in range(n):
  for j in range(n):
    l = list()
    if j+n*i-1000 >= 0:
      l.append(j+n*i-1000)
    if j+n*i-1 >= 0:
      l.append(j+n*i-1)
    if j+n*i+1 <= 999999:
      l.append(j+n*i+1)
    if j+n*i+1000 <= 999999:
      l.append(j+n*i+1000)
    
    grafo.append(l)

###NEAREST NEIGHBOR

In [72]:
pts_almacenes = []
pts_entregar = []

In [73]:
for i in range(n*n):
  if puntos[i][2] == 1:
    pts_almacenes.append(i)
  elif puntos[i][2] == 2:
    pts_entregar.append(i)

In [74]:
def clusters(warehouse_points, delivery_points): # La funcion clusters pone los puntos de entrega al punto de almacenes más cercano  
  manhattan = lambda a,b: abs(puntos[a][0]-puntos[b][0])+abs(puntos[a][1]-puntos[b][1])
  cl = []
  for i in range(len(warehouse_points)):
    cl.append([])

  for j in range(len(delivery_points)):
    d = delivery_points[j]
    mn = math.inf
    idmn = 0
    for i in range(len(warehouse_points)):
      w = warehouse_points[i]
      dis = manhattan(d,w) #se obtiene la distancia de cada warehouse_points hacia las los delivery_points
      if(dis < mn):
        mn = dis
        idmn = i
    
    cl[idmn].append(d)
  
  return cl

In [77]:
clusters_ = clusters(pts_almacenes,pts_entregar)
print(clusters_)

[[2716, 3619, 3706, 4603, 4678, 4721, 5580, 5598, 6563, 9663, 9670, 9713, 12733, 15647, 17712, 18523, 19621, 20651, 22713, 24545, 27635, 28562, 28564, 34577, 35626, 37657, 37683, 38601, 38635, 38687, 40633, 41618, 42692, 42699, 48634, 51692, 52601, 52611, 58619, 59671, 63614, 64676, 68698, 72696, 74619, 79661, 80679], [918, 2936, 8855, 8964, 16868, 18874, 18904, 18942, 21845, 22921, 32857, 33922, 34851, 34858, 34874, 34941, 35891, 41987, 42920, 43901, 43908, 46984, 48957, 48999, 51979, 52977, 59975, 64990, 67996, 74982, 77948, 78950, 87953, 88973], [9823, 11785, 15801, 18780, 20826, 21732, 22826, 28827, 30834, 33723, 34752, 35789, 35810, 46852, 48845, 50837, 54807, 55772, 58829, 62817, 65788, 67773, 69801, 69811, 73781, 73794, 74749, 79778, 83775], [397, 2420, 4337, 6335, 9364, 14427, 17346, 18405, 19335, 19379, 22367, 25346, 25425, 26392, 39334, 42331, 44330, 46363, 46390, 48326, 49392, 54397, 55362, 62345, 63391, 74394, 74400, 74421, 75425, 77368, 77389, 78327, 80354, 81420, 83336, 8

In [76]:
def give(clst,orden_pnt_almacen):
  new = pts_almacenes[orden_pnt_almacen]
  for i in range(len(clst[orden_pnt_almacen])):
    minima_distancia = math.inf
    for punto_entrega in clst[orden_pnt_almacen]:
      if puntos[punto_entrega][3]==-1:
        distancia = abs(puntos[punto_entrega][0]-puntos[new][0])+abs(puntos[punto_entrega][1]-puntos[new][1])

        if distancia < minima_distancia:
          minima_distancia = distancia
          ultimo_punto_entrega = punto_entrega

    puntos[ultimo_punto_entrega][3] = new
    new = ultimo_punto_entrega

In [78]:
for cluster in range(int(len(clusters_)/2+1),int(len(clusters_)*3/4)): #DEL CLUSTER 51 AL 75
  give(clusters_,cluster)

###RESULTADO FINAL

In [88]:
print(" PUNTO\t   X\tY  TIPO PADRE")
for i in range(int(len(clusters_)/2+1),int(len(clusters_)*3/4)): # INFO DE PUNTOS DE ENTREGA DE CLUSTERS 51 A 75
  for point in clusters_[i]:
    print(point,":",puntos[point])

 PUNTO	   X	Y  TIPO PADRE
501588 : [588, 501, 2, 524588]
501613 : [613, 501, 2, 501588]
506552 : [552, 506, 2, 509571]
509571 : [571, 509, 2, 514574]
513549 : [549, 513, 2, 506552]
520614 : [614, 520, 2, 501613]
524588 : [588, 524, 2, 536576]
532543 : [543, 532, 2, 532546]
532546 : [546, 532, 2, 513549]
536576 : [576, 536, 2, 546561]
540596 : [596, 540, 2, 545598]
542617 : [617, 542, 2, 520614]
545598 : [598, 545, 2, 555602]
546561 : [561, 546, 2, 532543]
554611 : [611, 554, 2, 554616]
554616 : [616, 554, 2, 542617]
555602 : [602, 555, 2, 554611]
562589 : [589, 562, 2, 564594]
564594 : [594, 564, 2, 540596]
564613 : [613, 564, 2, 570612]
565582 : [582, 565, 2, 562589]
570612 : [612, 570, 2, 574597]
570622 : [622, 570, 2, 576615]
573558 : [558, 573, 2, 570622]
574597 : [597, 574, 2, 565582]
576615 : [615, 576, 2, 564613]
600564 : [564, 600, 2, 573558]
465093 : [93, 465, 2, 477089]
469079 : [79, 469, 2, 465093]
477089 : [89, 477, 2, 485084]
477120 : [120, 477, 2, 489100]
482131 : [131, 4

###DATOS EXTRA

In [89]:
cantidades = []
for cluster in clusters_:
  cantidades.append(len(cluster))

In [90]:
from statistics import mean
from statistics import median
from statistics import mode

In [91]:
print("MAX:",max(cantidades))
print("MIN:",min(cantidades))
print("MEDIA:",mean(cantidades))
print("MEDIANA:",median(cantidades))
print("MODA:",mode(cantidades))

MAX: 123
MIN: 4
MEDIA: 34.95
MEDIANA: 31.0
MODA: 29
